# Introdução aos Modelos de Linguagem

Os **modelos de linguagem** são algoritmos que têm como objetivo processar, entender e gerar linguagem natural. Eles são treinados em grandes conjuntos de dados textuais e utilizam técnicas de aprendizado de máquina, como **redes neurais** profundas, para identificar padrões e estruturas na linguagem. Esses modelos têm diversas aplicações, como:

- **Tradução automática**: Converter textos de uma língua para outra.
- **Geração de texto**: Criar novos textos a partir de um prompt inicial.
- **Análise de sentimentos**: Identificar emoções ou opiniões expressas em textos.
- **Assistentes virtuais**: Como chatbots e sistemas de resposta automatizada.

A partir de arquiteturas avançadas, como o **Transformer** (base do modelo GPT), os modelos de linguagem modernos conseguem produzir textos cada vez mais coerentes e contextualmente relevantes, sendo capazes de lidar com tarefas de conversação, resposta a perguntas, e até mesmo de criação literária.

# LLaMA

O **LLaMA** (*Large Language Model Meta AI*) é uma família de modelos de linguagem desenvolvida pela **Meta AI** com o objetivo de fornecer um modelo poderoso e eficiente para tarefas de processamento de linguagem natural (PLN). Ele foi projetado para ser altamente escalável, permitindo que modelos menores atinjam um desempenho competitivo com grandes modelos de linguagem, como o GPT-3.

Algumas das características do LLaMA incluem:

- **Arquitetura baseada em transformadores**: Seguindo a estrutura popularizada por modelos como o GPT, o LLaMA utiliza camadas de transformadores para modelar a linguagem.
- **Eficiência**: O LLaMA foi otimizado para ser mais eficiente, o que significa que modelos menores podem ser utilizados para realizar tarefas complexas de PLN com menos recursos computacionais.
- **Tamanhos de modelo**: Ele está disponível em diferentes tamanhos, variando de 7B a 65B parâmetros, tornando-o acessível para diversas aplicações, desde dispositivos com menos poder de processamento até grandes servidores.

O LLaMA foi desenvolvido para ser utilizado em uma ampla variedade de tarefas, incluindo:

- **Geração de texto**: Criação de respostas e textos a partir de um prompt.
- **Tradução de linguagem natural**: Conversão de texto de uma língua para outra.
- **Resposta a perguntas**: Fornecer respostas contextualmente relevantes para questões específicas.




In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # baixar api do ollama
from IPython.display import clear_output

# Iniciar a API Ollama em uma thread separada 

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [ ]:
!pip install -U lightrag[ollama] # instalando a biblioteca AdalFlow

In [ ]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

In [ ]:
qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

In [ ]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)
output=qa("o que é uma equação diferencial?")
display(Markdown(f"**Answer:** {output.data}"))